<a href="https://colab.research.google.com/github/ccosmin97/IDC-Prostate_segmentation/blob/main/OHIF_TEST_DCM2NIIX_idc_prostate_segmentation_prostate05_qin_rep_repeat_export_OHIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prostate segmentation on IDC collection -- Qin-Prostate-Repeatablity
*   Dataset : [Qin-Prostate-Repeatability](https://portal.imaging.datacommons.cancer.gov/explore/filters/?collection_id=qin_prostate_repeatability)
*   Goal : Prostate segmentation using Task005 Prostate nnU-net pre-trained model, T2 and ADC multi-modal input

# Auth login

In [1]:
import os
# dcm2niix 
!sudo apt-get install dcm2niix 
# cmd = 'dcm2niix -z y -m y -o %s  %s ' % (pat_dir_nifti_path, path_to_dicom_ct_folder)
# print(cmd)
# ret = os.system(cmd)
# print (ret)
# !pip install dcm2niix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libyaml-cpp0.5v5
Suggested packages:
  pigz
The following NEW packages will be installed:
  dcm2niix libyaml-cpp0.5v5
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 295 kB of archives.
After this operation, 1,105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libyaml-cpp0.5v5 amd64 0.5.2-4ubuntu1 [150 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dcm2niix amd64 1.0.20171215-1 [145 kB]
Fetched 295 kB in 0s (3,092 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/

##Colab

In [2]:
#colab 
from google.colab import auth
auth.authenticate_user()

## Drive setup for upload results

In [3]:
# import os
# #drive mount
# from google.colab import drive
# drive.mount('/content/drive')
# ##setup paths
# path_drive_root = "/content/drive/MyDrive/"
# drive_exp_root_name = "IDC" 
# data = "qin-prostate-repeatability"
# pre_trained_mod = "task005-Prostate"
# timestamp = "2022_07_12_ADC_2D"#year_month_day_MOD

In [4]:
# #personal drive path -- make sure folder exists in google drive
# path_experiment = os.path.join(path_drive_root,
#                                drive_exp_root_name,
#                                data,
#                                pre_trained_mod,
#                                timestamp)

# if not os.path.exists(path_experiment):
#   !mkdir -p $path_experiment

# Global variables

## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [5]:
# IDC collection for paths setup
collection_name = "qin_prostate_root_data" #up to user
##nnunet
model_type = '3d_fullres'#other options are '2d',..
nnunet_labelID = 1 #for task005 model, labelid=1 means PZ and 2 means TZ
##labelID seg retrieval
ground_truth_cat_CodeMeaning = 'Anatomical Structure'
ground_truth_type_CodeMeaning = 'Peripheral zone of the prostate'
#Resampling scheme -- this option is made for qin-rep-repeatability(SeriesDescription tag dependence,etc), not modular for other collections with multi modal inference input
desired_grid = "T2" #Other option is "ADC" 

## Global paths

In [6]:
import os 

#qin_prostate_repeatibility
os.environ["qin_prostate_rep_root"] = os.path.join(os.getcwd(), collection_name)
os.environ["qin_prostate_rep_dicom"] = os.path.join(os.environ["qin_prostate_rep_root"], "dicom")
os.environ["qin_prostate_rep_nii"] = os.path.join(os.environ["qin_prostate_rep_root"], "nii")
os.environ["qin_prostate_rep_nrrd"] = os.path.join(os.environ["qin_prostate_rep_root"], "nrrd")
os.environ["qin_prostate_rep_analysis"] = os.path.join(os.environ["qin_prostate_rep_root"], "results")
os.environ["qin_prostate_rep_analysis_data"] = os.path.join(os.environ["qin_prostate_rep_analysis"], "data_seg")
os.environ["qin_prostate_rep_analysis_verbose"] = os.path.join(os.environ["qin_prostate_rep_root"], "results_verbose")


#resampled data -- To t2 mod
os.environ["qin_prostate_rep_root_resampled"] = os.path.join(os.environ["qin_prostate_rep_root"], "resampled")
os.environ["qin_prostate_rep_nii_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nii")
os.environ["qin_prostate_rep_nrrd_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nrrd")

#nnunet
os.environ["nnUNet"] = os.path.join(os.getcwd(), "nnUNet")
os.environ["nnUNet_data"] = os.path.join(os.environ["nnUNet"], "data")
os.environ['nnUNet_raw_data_base'] = os.path.join(os.environ["nnUNet_data"], "nnUNet_raw_data")
os.environ['nnUNet_preprocessed'] = os.path.join(os.environ["nnUNet_data"], "processed")
os.environ["nnUNet_models"] = os.path.join(os.environ["nnUNet"], "models")
os.environ["RESULTS_FOLDER"] = os.path.join(os.environ["nnUNet"], "output", "preds")
os.environ["nnUNet_preds_resampled"] = os.path.join(os.environ["nnUNet"], "output", "preds_resampled")
os.environ["nnUNet_preds_resampled_dcm"] = os.path.join(os.environ["nnUNet"], "output", "preds_resampled_dcm")
os.environ["nnUNet_raw_data"] = os.path.join(os.environ["nnUNet"], "raw_data")
#path where model pre-trained weights are stored
os.environ["PATH_TO_MODEL_FILE"] = os.path.join(os.environ["nnUNet"], "models", "Task005_Prostate.zip")

#misc
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")
os.environ["logs"] = os.path.join(os.getcwd(), "logs")

#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["qin", "nnunet", "IDC", "nnUNet", "logs"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Setup GCP Project ID

In [7]:
project_id = "idc-nlst-unet-seg"
os.environ["GCP_PROJECT_ID"] = project_id

# Download nnUnet pre-trained model on prostate decathlon data -- task005 -- personal dropbox

nnUnet pre-trained models zip files can also be found [here](https://zenodo.org/record/4003545#.YsWmH-zMLt8).

In [8]:
print(os.environ["nnUNet_models"])

/content/nnUNet/models


In [9]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_model_url = "https://www.dropbox.com/s/igpwt45v6hlquxp/Task005_Prostate.zip?dl=0"
out_path_mod = os.path.join(os.environ["nnUNet_models"], "Task005_Prostate.zip")#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $out_path_mod $seg_model_url

--2022-10-04 19:56:06--  https://www.dropbox.com/s/igpwt45v6hlquxp/Task005_Prostate.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/igpwt45v6hlquxp/Task005_Prostate.zip [following]
--2022-10-04 19:56:07--  https://www.dropbox.com/s/raw/igpwt45v6hlquxp/Task005_Prostate.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7b6d7b2168ef4d7280e3fc6383.dl.dropboxusercontent.com/cd/0/inline/BuN87mgldfeDwQ74q9mcSF-dSyuCg3V5_d9HrhOn-e9IRpZ3xK7xlxvJFo16WhemDXCIrh8hnmT58RiDSYddLPypbqTUkOHdk9YYb8rrmCi3n3xHd98mdsTShm6C3nCLjyT9oTHt3OH3LhQu2o71qiwS4Mwzn-pcLDW1BkybLP_YjA/file# [following]
--2022-10-04 19:56:07--  https://uc7b6d7b2168ef4d7280e3fc6383.dl.dropboxusercontent.com/cd/0/inline/BuN87mgldfeDwQ74q9mcSF-dSyuCg3V5_d9HrhOn-e9IRpZ3xK7

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies. 

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

In [10]:
# check wether the Colab Instance was correctly initialized with a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [11]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Tue Oct  4 19:56:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Environment Setup

Here we will configure the Linux environment variables needed to run the nnU-Net pipeline. 

Three main variables are needed by default to run the nnU-Net segmentation pipelines:
* `nnUNet_raw_data_base` is the path to the folder where the segmentation pipeline expects to find the data to process;
* `nnUNet_preprocessed` is the path to the folder where the preprocessed data are saved;
* `RESULTS_FOLDER` is the path to the folder storing by default the model weights and, in our case, for simplicity, the segmentation masks produced by the pipeline.

We will use the additional variable `PATH_TO_MODEL_FILE` to point to the location where the pre-trained model weights for the chosen model will be stored (more on this later).

Please notice that these variables need to be set using `os.environ[]` in Google Colab - as `!export` is not sufficient to guarantee the variables are kept from one cell to the other. For more in-depth information regarding what the nnU-Net framework uses these folders for, please visit [the dedicated nnU-Net documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md)

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [12]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [13]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [14]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [15]:
%%capture

!sudo apt install subversion

In [16]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled May 21 2022, 07:24:25 on x86_64-pc-linux-gnu


## Install Python packages

In [17]:
%%capture
!pip install nnunet
!pip install pydicom
!pip install nibabel
!pip install dcm2niix

Unpack and install model we downloaded earlier (under `PATH_TO_MODEL_FILE`). This step can take about 1-2 minutes.

In [18]:
%%capture
!nnUNet_install_pretrained_model_from_zip $PATH_TO_MODEL_FILE

Next we set up few things to help with visualization of the segmentations later.

In [19]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

from medpy.metric.binary import dc as dice_coef
from medpy.metric.binary import hd as hausdorff_distance
from medpy.metric.binary import asd as avg_surf_distance

from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
%tensorflow_version 2.x
import tensorflow as tf
import keras

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)
print("TensorFlow version           : ", tf.__version__)
print("Keras (stand-alone) version  : ", keras.__version__)

print("\nThis Colab instance is equipped with a GPU.")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Python version               :  3.7.14 (default, Sep  8 2022, 00:06:44) 
Numpy version                :  1.21.6
TensorFlow version           :  2.8.2
Keras (stand-alone) version  :  2.8.0

This Colab instance is equipped with a GPU.


In [20]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

A    pyplastimatch/__init__.py
A    pyplastimatch/pyplastimatch.py
A    pyplastimatch/utils
A    pyplastimatch/utils/__init__.py
A    pyplastimatch/utils/data.py
A    pyplastimatch/utils/eval.py
A    pyplastimatch/utils/viz.py
Checked out revision 25.


In [21]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 11.03 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [22]:
from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.utils import viz as viz_utils
from pyplastimatch.utils import data as data_utils

# Data selection, downloading and structuring

We want to select here the collection named qin-prostate repeatibility, and more particularly the two timepoints per patient ID for further analysis.

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

## Data selection -- BigQuery

Explanation

In [23]:
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])
selection_query = f"""
  SELECT  
  StudyInstanceUID, 
  SeriesInstanceUID, 
  SOPInstanceUID, 
  PatientID, 
  Modality, 
  collection_id, 
  StudyDescription, 
  SeriesDescription, 
  StudyDate, 
  SeriesDate,
  gcs_url,
  (SELECT SeriesInstanceUID FROM UNNEST(ReferencedSeriesSequence)) AS RefSerieUID 
FROM 
  `bigquery-public-data.idc_current.dicom_all` 
WHERE 
  collection_id = 'qin_prostate_repeatability' 
  AND SeriesDescription IN ('T2 Weighted Axial', 'Apparent Diffusion Coefficient', 
    'T2 Weighted Axial Segmentations', 'Apparent Diffusion Coefficient Segmentations')""" 

In [24]:
selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

## Download gs urls into dcm files and sorting  

### Randomly pick X PatientIDs for the use case - pick 5 by default

In [26]:
# # # #random PatientID selection
# # patientID_select = random.sample(list(selection_df.PatientID.values), 5)#X randomly picked PatientIDs 

# # static PatientID selection
patientID_select = ["PCAMPMRI-00007", "PCAMPMRI-00006", "PCAMPMRI-00009", "PCAMPMRI-00013", "PCAMPMRI-00015"]

selection_df = selection_df[selection_df.PatientID.isin(patientID_select)]

In [27]:
selection_df.PatientID.unique()

array(['PCAMPMRI-00006', 'PCAMPMRI-00013', 'PCAMPMRI-00007',
       'PCAMPMRI-00009', 'PCAMPMRI-00015'], dtype=object)

In [28]:
selection_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 554 entries, 0 to 1563
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   StudyInstanceUID   554 non-null    object
 1   SeriesInstanceUID  554 non-null    object
 2   SOPInstanceUID     554 non-null    object
 3   PatientID          554 non-null    object
 4   Modality           554 non-null    object
 5   collection_id      554 non-null    object
 6   StudyDescription   554 non-null    object
 7   SeriesDescription  554 non-null    object
 8   StudyDate          554 non-null    object
 9   SeriesDate         554 non-null    object
 10  gcs_url            554 non-null    object
 11  RefSerieUID        20 non-null     object
dtypes: object(12)
memory usage: 56.3+ KB


### Download

In [35]:
# save the list of GCS URLs into a file
selection_manifest = os.path.join(os.environ["IDC_Downloads"], "idc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [36]:
# confirm the resulting manifest has as many lines as the number of rows in the
# dataframe we initialized earlier
!cat {selection_manifest} |wc

    554     554   36564


In [37]:
# let's make sure the download folder is clean, in case you ran this cell earlier
# for a different dataset
!rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}

In [38]:
# download is this simple!
%%capture

!cat {selection_manifest} | gsutil -m cp -I {os.environ["IDC_Downloads"]}

To facilitate preparation of the data, we will sort the downloaded files into the PatientID/StudyInstanceUID/SeriesInstanceUID hierarchy.

In [39]:
!python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 555/555 [00:03<00:00, 168.16it/s]
Files sorted


Move the sorted data into the right place

In [40]:
rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}* && mv {os.environ["IDC_Downloads_Sorted"]+"/*"} {os.environ["qin_prostate_rep_dicom"]}

#Pre-Processing

Dicom files are assumed to be sorted this way for further processing :

* PatientID
  * StudyUID
    * SeriesUID
      * *.dcm file(s)

## conversion methods

https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24283641

switch from plastimatch resample method to simple itk for slice spacing issue

In [41]:
def safe_sitk_read(img_list, *args, **kwargs):
  """
  Since the default function just looks at images 0 and 1 to determine slice thickness
  and the images are often not correctly alphabetically sorted, much slower
  :param img_list:
  :return:
  """
  pimg_list = [(sitk.ReadImage(x).GetOrigin(), x) for x in img_list]
  s_img_list = [path for _, path in sorted(pimg_list, key = lambda x: x[0][2])] # sort by z
  return sitk.ReadImage(s_img_list, *args, **kwargs)

In [61]:
def convert_mr_to_nii_nrrd(input_path, output_path_root, modality, target_format="nii.gz"):
  output_path = os.path.join(output_path_root+"_"+modality)#,glob.glob(os.path.join(input_path, "**", "*.dcm"), recursive=True)[0].split('/')[-2]+".nii.gz")
  if not os.path.exists(output_path): 
    !mkdir -p $output_path
  !dcm2niix -z y -m y -o $output_path $input_path
  # dcm_img = safe_sitk_read(glob.glob(os.path.join(input_path, "**", "*.dcm"), recursive=True))
  # sitk.WriteImage(dcm_img, output_path)

We will use the [segimage2itkimage tool](https://qiicr.gitbook.io/dcmqi-guide/opening/cmd_tools/seg/segimage2itkimage) from dcmqi to extract individual segments from DICOM SEG series we just downloaded into NII format, saving each segment into a separate file

In [62]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path): 
    !mkdir -p $output_path
  
  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii 

In [63]:
def convert_dcm_sorted(input_path,output_path):
  path_dicom = os.environ["qin_prostate_rep_dicom"] 
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#, recursive = True):
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    modality = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#'SEG' if pydicom.dcmread(path_serie_dcm_lst[0]).Modality == "SEG" else "MR"
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    SeriesDescription = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["SeriesDescription"].iloc[0]
    type_serie = ["T2" if "T2" in SeriesDescription else "ADC"][0]
    if modality == "SEG":
      #convert to nrrd
      convert_seg_to_nii(input_path=path_serie_dcm_lst[0],
                              output_path=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_{}_".format(type_serie) +"SEG"))
    elif modality == "MR":
      #convert to nii
      convert_mr_to_nii_nrrd(input_path=serie_folder,
                              output_path_root=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nii.gz")
      #convert to nrrd
      convert_mr_to_nii_nrrd(input_path=serie_folder, 
                              output_path_root=os.path.join(output_path, "nrrd", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nrrd")
    else:
      print(f"type serie {modality} not supported")

## Conversion to NII/NRRD

In [64]:
!rm -rf /content/qin_prostate_root_data/nii/*
!rm -rf /content/qin_prostate_root_data/nrrd/*

In [65]:
convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
                   output_path=os.environ["qin_prostate_rep_root"])

input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846371.12016.1513205145.257/1.2.276.0.7230010.3.1.4.1426846371.12016.1513205145.258.dcm
output_path : /content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846371.12016.1513205145.257_ADC_SEG
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
W: CodeValue (0008,0100) empty in CodeSequenceMacro (type 1C)
Row direction: 0.999991 0.00077563 0.0041239
Col direction: -0.00204531 0.948324 0.317298
Z direction: -0.00366468 -0.317304 0.948317
Total frames: 72
Total frames with unique IPP: 24
Total overlapping frames: 24
Origin: [-103.603, -137.082, -99.4545]
input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846

# Resampling ground truth segmentations to desired grid

In [66]:
!rm -rf /content/qin_prostate_root_data/resampled/nii/*
!rm -rf /content/qin_prostate_root_data/resampled/nrrd/*

In [67]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*_SEG", "*.nii.gz"), recursive=True)
for seg_nii in sorted(nii_lst) :
  print(seg_nii)
  patientID = seg_nii.split('/')[-4]
  studyUID = seg_nii.split('/')[-3]
  serieUID = seg_nii.split('/')[-2].split('_')[0] #standard serieUID without T2_SEG custom suffix
  type_serie = '_'.join(seg_nii.split('/')[-2].split('_')[1:])
  #get referencedSerieUID from seg corresponding to T2 MR volume 
  referencedSerieUID = str(selection_df[selection_df["SeriesInstanceUID"] == serieUID]["RefSerieUID"].iloc[0])	
  matched_mod = glob.glob(os.path.join(os.environ['qin_prostate_rep_nii'], 
                                            patientID, studyUID, referencedSerieUID + "_MR*", "*.nii*"))[0]
  output_path_seg_root = os.path.join(os.environ["qin_prostate_rep_nii_resampled"],
                                  patientID, studyUID,
                                  serieUID + f"_{type_serie}")
  if not os.path.exists(output_path_seg_root):
    !mkdir -p $output_path_seg_root
  resample_args = {"input" : seg_nii, 
                        "output" : os.path.join(output_path_seg_root, seg_nii.split('/')[-1]),
                        "fixed" : matched_mod,
                        "interpolation" : "nn",
                   "logfile" : os.path.join(output_path_seg_root, seg_nii.split('/')[-1][:-7]+".log")}
  #resample to t2 mod based on study -- not uniform across studies
  pypla.resample(verbose = True, **resample_args)

/content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846371.12016.1513205145.257_ADC_SEG/1.nii.gz

Running 'plastimatch resample' with the specified arguments:
  --input /content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846371.12016.1513205145.257_ADC_SEG/1.nii.gz
  --output /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.2.276.0.7230010.3.1.3.1426846371.12016.1513205145.257_ADC_SEG/1.nii.gz
  --fixed /content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.3.6.1.4.1.14519.5.2.1.3671.4754.882539338600550863063890292531_MR_ADC/1.3.6.1.4.1.14519.5.2.1.3671.4754.882539338600550863063890292531_PROSTATE_DCE_2013_19941001133534_901.nii.gz
  --interpolation nn
  --logfile 

# Resampling input data(T2--ADC) to desired T2 or ADC resampling grid

In [68]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*_MR*", "*.nii.gz"), recursive=True)
for mr_nii in sorted(nii_lst) :
  # print(mr_nii)
  patientID = mr_nii.split('/')[-4]
  studyUID = mr_nii.split('/')[-3]
  serieUID = mr_nii.split('/')[-2].split('_')[0] #standard serieUID without T2_SEG custom suffix
  sopInstanceUID = mr_nii.split('/')[-1]
  type_serie = mr_nii.split('/')[-2].split('_')[-1]#.split('.')[0]
  print(f'type_serie : {type_serie}')
  print(f'serieUID : {serieUID}')
  output_path_root = os.path.join(os.environ["qin_prostate_rep_nii_resampled"],
                                  patientID, studyUID, mr_nii.split('/')[-2]) 
  # output_path = os.path.join(output_path_root,sopInstanceUID) 
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  if desired_grid != type_serie:
    print("need to resample")
    match_des_grid = glob.glob(os.path.join(os.environ['qin_prostate_rep_nii'],
                                         patientID, studyUID, 
                                         f"*_MR_{desired_grid}", #serieUID
                                         "*.nii*"))[0] #SOPUID
    print(f"match_des_grid : {match_des_grid}")
    resample_args = {"input" : mr_nii, 
                        "output" : os.path.join(output_path_root, mr_nii.split('/')[-1]),
                        "fixed" : match_des_grid}
    #resample based on study -- not uniform across studies
    pypla.resample(verbose = False, **resample_args)
  elif desired_grid == type_serie :#just cp other volume for now, no fixed resampling grid option implemented yet
    !cp $mr_nii $output_path_root
  else:
    pass

type_serie : T2
serieUID : 1.3.6.1.4.1.14519.5.2.1.3671.4754.330595492110314365252888110460
type_serie : ADC
serieUID : 1.3.6.1.4.1.14519.5.2.1.3671.4754.882539338600550863063890292531
need to resample
match_des_grid : /content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.3.6.1.4.1.14519.5.2.1.3671.4754.330595492110314365252888110460_MR_T2/1.3.6.1.4.1.14519.5.2.1.3671.4754.330595492110314365252888110460_PROSTATE_DCE_2013_19941001133534_6.nii.gz
type_serie : T2
serieUID : 1.3.6.1.4.1.14519.5.2.1.3671.4754.124813816937601110554342446480
type_serie : ADC
serieUID : 1.3.6.1.4.1.14519.5.2.1.3671.4754.851354669834028302840216094576
need to resample
match_des_grid : /content/qin_prostate_root_data/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.287814201869490350714498330797/1.3.6.1.4.1.14519.5.2.1.3671.4754.124813816937601110554342446480_MR_T2/1.3.6.1.4.1.14519.5.2.1.3671.4754.124813816937601110554342446480_PROSTATE_DCE_201

# Inference -- nnUnet

## Naming Formatting files for nnUnet inference

In [69]:
t2_list = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii_resampled"], 
                                 "**", "**", "*_MR_T2", f"*.nii.gz"), recursive=True)
adc_list = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii_resampled"], 
                                 "**", "**", "*_MR_ADC", f"*nii.gz"), recursive=True)
matchStudyUID = lambda x: (x[0], x[1]) if x[0].split('/')[-3] == x[1].split('/')[-3] else None
sorted_mr_lst = [matchStudyUID((t2_mr, adc_mr)) 
                    for t2_mr in t2_list 
                        for adc_mr in adc_list 
                            if matchStudyUID((t2_mr, adc_mr)) is not None]
#make sure studyUIDs match
for el in sorted_mr_lst : assert el[0].split('/')[-3] == el[1].split('/')[-3]
#name formatting and copying to target folder
for mr_t2_adc in sorted_mr_lst:
  for mr_vol in mr_t2_adc: 
    studyInstanceUID = mr_vol.split('/')[-3]
    patientID = mr_vol.split("/")[-4]
    nnunet_idx = "0000" if "T2" in mr_vol.split('/')[-2] else "0001"#0000 for T2 and 0001 for ADC
    nnunet_path = os.path.join(os.environ["nnUNet_preprocessed"], 
                                "_".join([patientID, studyInstanceUID, nnunet_idx]) + ".nii.gz") 
    !cp $mr_vol $nnunet_path

## Inference

In [70]:
!nnUNet_print_pretrained_model_info Task005_Prostate



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Prostate Segmentation. 
Segmentation targets are peripheral and central zone, 
Input modalities are 0: T2, 1: ADC. 
Also see Medical Segmentation Decathlon, http://medicaldecathlon.com/


In [71]:
print(f"input folder path : "+os.environ["nnUNet_preprocessed"])
print(f"output folder path : "+os.environ["RESULTS_FOLDER"])

input folder path : /content/nnUNet/data/processed
output folder path : /content/nnUNet/output/preds


In [72]:
!echo {os.environ["nnUNet_preprocessed"]} 

/content/nnUNet/data/processed


In [73]:
%%capture
!nnUNet_install_pretrained_model_from_zip $out_path_mod

In [74]:
!nnUNet_predict --input_folder {os.environ["nnUNet_preprocessed"]} \
                --output_folder {os.environ["RESULTS_FOLDER"]} \
                --task_name "Task005_Prostate" --model $model_type \
                --save_npz 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/nnUNet/output/preds/nnUNet/3d_fullres/Task005_Prostate/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 10 unique case ids, here are some examples: ['PCAMPMRI-00015_1.3.6.1.4.1.14519.5.2.1.3671.4754.182983245842036631781777324507'
 'PCAMPMRI-00006_1.3.6.1.4.1.14519.5.2.1.3671.4754.287814201869490350714498330797'
 'PCAMPMRI-00015_1.3.6.1.4.1.14519.5.2.1.3671.4754.182983245842036631781777324507'
 'PCAMPMRI-00007_1.3.6.1.4.1.14519.5.2.1.3671.4754.240243858523310211919840278205'
 'PCAMPMRI-00009_1.3.6.1.4.1.14519.5.2.1.3671.4754.181478254003851548055481596513'


# Resamplings preds back to input data -- desired grid

In [75]:
for pred_path in sorted(glob.glob(os.path.join(os.environ["RESULTS_FOLDER"], "*.nii.gz"))):
  search_t2_path = os.path.join(os.environ["qin_prostate_rep_nii_resampled"], pred_path.split('/')[-1].split('_')[0], #get PatientID
                                   pred_path.split('/')[-1].split('_')[-1][:-7], #get studyUID
                                    f"*{desired_grid}", #SerieUID
                                   "*.nii.gz") #dcm2niix file format
  print(f"search path for {desired_grid} : {search_t2_path}")
  t2_path = glob.glob(search_t2_path, recursive=True)[0]                              
  print(f"{desired_grid}_path found : {t2_path}")
  print(f"pred path : {pred_path}")
  resample_args_to_t2_origin = {"input" : pred_path, 
                        "output" : os.path.join(os.environ["nnUNet_preds_resampled"], 
                                                pred_path.split('/')[-1]),
                        "fixed" : t2_path,
                        "interpolation" : "nn"}
  
  path_log = os.path.join(os.environ["logs"], 'log_pypla_res_pred' + pred_path.split('/')[-1].split('.')[0] + '.txt')            
  !touch $path_log
  pypla.resample(verbose = False, **resample_args_to_t2_origin, path_to_log_file=path_log)
  print()

search path for T2 : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/*T2/*.nii.gz
T2_path found : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398/1.3.6.1.4.1.14519.5.2.1.3671.4754.330595492110314365252888110460_MR_T2/1.3.6.1.4.1.14519.5.2.1.3671.4754.330595492110314365252888110460_PROSTATE_DCE_2013_19941001133534_6.nii.gz
pred path : /content/nnUNet/output/preds/PCAMPMRI-00006_1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398.nii.gz

search path for T2 : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.287814201869490350714498330797/*T2/*.nii.gz
T2_path found : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00006/1.3.6.1.4.1.14519.5.2.1.3671.4754.287814201869490350714498330797/1.3.6.1.4.1.14519.5.2.1.3671.4754.124813816937601110554342446480_MR_T2/1.3.6.1.4.1.14519.

# EXPORT PREDS TO DCM

Get template from dropbox for whole prostate gland 

-----

try manually then automate !


---

 with resampled preds and seg gts and t2 vols


In [76]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_whole_prostate_json = "https://www.dropbox.com/s/y11eg3ek4gu8p4i/seg-example_multiple_segments_single_input_file.json?dl=0"
seg_dcm_metadata_json_file = "pred_metadata.json"#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $seg_dcm_metadata_json_file $seg_whole_prostate_json

--2022-10-04 20:17:15--  https://www.dropbox.com/s/y11eg3ek4gu8p4i/seg-example_multiple_segments_single_input_file.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/y11eg3ek4gu8p4i/seg-example_multiple_segments_single_input_file.json [following]
--2022-10-04 20:17:15--  https://www.dropbox.com/s/raw/y11eg3ek4gu8p4i/seg-example_multiple_segments_single_input_file.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc473fdeecebec3f6247eaa075ab.dl.dropboxusercontent.com/cd/0/inline/BuOBjmg4JvaDD2qaGHgR5MTvafxzNUDxRnSyT68qR3LS-TYLeW1dB21a8ckxksRSJALF_4jxUyUjgnQpio-uq8hqNDRgtiItVmIpmf4VckOT7MsFvhItnWpvQkU5Li7D7zBq5i3vwcWQ1gaWtSlSwdm_ISQ1tdwZGAu3Zc4lt7pA8Q/file# [following]
--2022-10-04 20:17:16--  https://uc473fdeecebec3f6247ea

In [77]:
for nii_seg_pred in glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], '*.nii.gz')):
  patID = nii_seg_pred.split('/')[-1].split('_')[0]
  study_current = nii_seg_pred.split('/')[-1].split('_')[1][:-7]

  study_mr_t2_serieUID = selection_df[(selection_df["StudyInstanceUID"] == study_current) & (selection_df["Modality"] == "MR") \
                                      & (selection_df["SeriesDescription"].str.contains('T2', na=False))]["SeriesInstanceUID"].unique()[0]
  study_seg_serieUID = selection_df[(selection_df["StudyInstanceUID"] == study_current) & (selection_df["Modality"] == "SEG") \
                                      & (selection_df["RefSerieUID"].str.contains(study_mr_t2_serieUID, na=False))]["SeriesInstanceUID"].unique()[0]
  #find t2 dcm folder
  t2_dcm_folder = os.path.join(os.environ["qin_prostate_rep_dicom"], patID, study_current, study_mr_t2_serieUID)
  #find seg dcm file
  nii_seg_folder = os.path.join(os.environ["qin_prostate_rep_dicom"], patID, study_current, study_seg_serieUID)
  # find nii seg folder == preds resampled
  assert os.path.exists(t2_dcm_folder)
  assert os.path.exists(nii_seg_folder)
  print('\nConverting...')
  print(f'pred nnunet procssed : {nii_seg_pred}')
  print(f't2_dcm_folder : {t2_dcm_folder}')
  print(f'nii_seg_folder : {nii_seg_folder}')
  output_path = os.path.join(os.environ["nnUNet_preds_resampled_dcm"], '_'.join([patID,study_current,study_seg_serieUID])+'.dcm')
  #find gt seg dcm file == orginal idc dcm files
  #convert nii pred to dcm
  !itkimage2segimage --inputImageList $nii_seg_pred \
  --inputDICOMDirectory  $t2_dcm_folder \
  --inputMetadata  $seg_dcm_metadata_json_file \
  --outputDICOM $output_path \
  --skip
  print("Done!")


Converting...
pred nnunet procssed : /content/nnUNet/output/preds_resampled/PCAMPMRI-00009_1.3.6.1.4.1.14519.5.2.1.3671.4754.864607788085827202303582199118.nii.gz
t2_dcm_folder : /content/qin_prostate_root_data/dicom/PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.864607788085827202303582199118/1.3.6.1.4.1.14519.5.2.1.3671.4754.117459950571016749056636056931
nii_seg_folder : /content/qin_prostate_root_data/dicom/PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.864607788085827202303582199118/1.2.276.0.7230010.3.1.3.1426846371.13920.1513205159.361
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Loaded segmentation from /content/nnUNet/output/preds_resampled/PCAMPMRI-00009_1.3.6.1.4.1.14519.5.2.1.3671.4754.864607788085827202303582199118.nii.gz
Searching recursively /content/qin_prostate_root_data/dicom/PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.864607788085827202303582199118/1.3.6.1.4.1.14519.5.2.1.3671.4754.117459950571016749056636056931 for DIC

# EXPORT TO OHIF 

## User input 

Set these values. 

In [97]:
# project_name = 'idc-sandbox-003'
# location_id = 'us-central1'
# my_bucket = 'nnunet_seg_test01_bucket'
# my_dataset = 'qin-prost-rep_task024_dataset'
# my_datastore = 'qin-prost-rep_task024_datastore'

project_name = 'idc-sandbox-003'
location_id = 'us-central1'
my_bucket_seg_pred = 'prostate-seg-use-case/data/qin_prostate_repeatability/seg_pred/task05/'
my_bucket_seg_gt = 'prostate-seg-use-case/data/qin_prostate_repeatability/seg_gt/'
my_bucket_data = 'prostate-seg-use-case/data/qin_prostate_repeatability/data_in/'
my_dataset = 'prostate-seg'
my_datastore = 'test1_datastore_dcm2niix'

# GCP Healthcare steps

In order to view the MR data along with the SEGS, we need to store our data in the cloud. We first need to create a dataset, and a DICOM datastore, which will hold our MR and SEG data. To do this we will first store our data in a bucket, and then copy over the contents to the DICOM datastore. 

We first need to create a dataset if it doesn't already exist. 

In [80]:
# First let's list the datasets that we already have for our particular project_id and location_id
datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))

datasets that exist for project_id idc-sandbox-003, location us-central1: ['ct_lymph_tcia', 'nnunet_seg_test01', 'ohif-tests', 'prostate-seg', 'qin-prost-rep_task024_dataset']


Create a dicom store if it doesn't already exist

In [81]:
# # First list the datastores that exist in the dataset
datastores = !gcloud healthcare dicom-stores list --project $project_name --dataset $my_dataset #--format="value(ID)"
print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(my_dataset) + ': ' + str(datastores))

datastores that exist for project_id idc-sandbox-003, location us-central1, dataset prostate-seg: ['ID                        LOCATION     TOPIC', 'test1_datastore_dcm2niix  us-central1']


In [98]:
# COPY DATA + GT SEG
!gsutil -m cp -r {os.environ["qin_prostate_rep_dicom"]+'/*'} {'gs://'+my_bucket_data}
# COPY PREDS
!gsutil -m cp -r {os.environ["nnUNet_preds_resampled_dcm"]+'/*'} {'gs://'+my_bucket_seg_pred}

Copying file:///content/nnUNet/output/preds_resampled_dcm/PCAMPMRI-00006_1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398_1.2.276.0.7230010.3.1.3.1426846371.2776.1513205144.321.dcm [Content-Type=application/dicom]...
Copying file:///content/nnUNet/output/preds_resampled_dcm/PCAMPMRI-00007_1.3.6.1.4.1.14519.5.2.1.3671.4754.240243858523310211919840278205_1.2.276.0.7230010.3.1.3.1426846371.11036.1513205146.731.dcm [Content-Type=application/dicom]...
Copying file:///content/nnUNet/output/preds_resampled_dcm/PCAMPMRI-00006_1.3.6.1.4.1.14519.5.2.1.3671.4754.287814201869490350714498330797_1.2.276.0.7230010.3.1.3.1426846371.15788.1513205142.615.dcm [Content-Type=application/dicom]...
Copying file:///content/nnUNet/output/preds_resampled_dcm/PCAMPMRI-00007_1.3.6.1.4.1.14519.5.2.1.3671.4754.962865348146140991373362412648_1.2.276.0.7230010.3.1.3.1426846371.20336.1513205149.477.dcm [Content-Type=application/dicom]...
Copying file:///content/nnUNet/output/preds_resampled_dcm/PCAMPMR

In [100]:
# Add MRI and SR from bucket to dicomstore 

!gcloud healthcare dicom-stores import gcs $my_datastore \
                                            --dataset=$my_dataset \
                                            --location=$location_id \
                                            --project=$project_name \
                                            --gcs-uri=gs://prostate-seg-use-case/data/qin_prostate_repeatability/**.dcm

Request issued for: [test1_datastore_dcm2niix]
name: projects/idc-sandbox-003/locations/us-central1/datasets/prostate-seg/dicomStores/test1_datastore_dcm2niix


Navigate to the OHIF app you created and open the DICOM datastore. 